In [ ]:
!pip install transformers soundfile torchmetrics gdown

In [ ]:
import gdown
def drive_download(idx, output):
    url = 'https://drive.google.com/uc?id=' + idx
    gdown.download(url, output, quiet=False)
drive_download("1ZepptsTrVSjQEx-dpBBmQ2b7xYFLn_64", "public_test.zip")

In [ ]:
# !unzip ./public_test.zip -d ./dataset

In [ ]:
# !cp -r ./drive/MyDrive/checkpoint/checkpoint_slu.pt ./

In [ ]:
import torch
import utils
from trainer import Trainer
from model import BertSLU
from functools import partial
from dataset import BertDataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

In [ ]:
def custom_collate(tokenizer, is_train, batch):
    inputs = tokenizer([i["text"] for i in batch], return_tensors="pt", padding="longest")
    if not is_train:
        return inputs, torch.zeros_like(inputs["input_ids"]), torch.zeros(inputs["input_ids"].size(0))
    seq_len = inputs["input_ids"].size(1)
    token_labels = torch.stack([
        torch.tensor(i["token_label"] + [-100]*(seq_len - len(i["token_label"]))) for i in batch
    ])
    intent_labels = torch.tensor([i["intent_label"] for i in batch])
    return inputs, token_labels, intent_labels

In [ ]:
def get_loader(annotation_path, token_label_path, batch_size=2, test_size=0.3):
    tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
    annotations = utils.load_annotation(annotation_path)
    all_intent = [i["intent"] for i in annotations]
    all_text = [i["sentence"] for i in annotations]
    all_label = utils.load_json(token_label_path)
    all_label = [all_label, all_intent]
    dataset = BertDataset(all_text, all_label, utils.MAP_INTENT)
    N = len(dataset)
    train_size = int(N * (1-test_size))
    train_set, valid_set = torch.utils.data.random_split(dataset, [train_size, N-train_size])
    train_loader = DataLoader(
        train_set,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=partial(custom_collate, tokenizer, True)
    )
    valid_loader = DataLoader(
        valid_set,
        batch_size=batch_size,
        collate_fn=partial(custom_collate, tokenizer, True)
    )
    return train_loader, valid_loader

In [ ]:
def get_test_loader(test_path, batch_size=2):
    tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
    sequences = utils.load_json(test_path)
    id_seqs = [k for k, v in sequences.items()]
    seqs = [v for k, v in sequences.items()]
    dataset = BertDataset(seqs)
    test_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=partial(custom_collate, tokenizer, False)
    )
    return test_loader, id_seqs, seqs

In [ ]:
train_loader, valid_loader = get_loader("./dataset/train.jsonl", "./dataset/bert_token_labels.json", 16)
print(f"Len train_loader: {len(train_loader)} - Len valid_loader: {len(valid_loader)}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Len train_loader: 328 - Len valid_loader: 141


In [ ]:
test_loader, test_file_id, all_seqs = get_test_loader("./dataset/test_sentences.json", 8)
len(test_loader)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


163

In [ ]:
class config:
    epochs = 20
    checkpoint_path = "./checkpoint/checkpoint_bert.pt"
    learning_rate = 5e-5
    adam_eps = 1e-8
    warmup_steps = 2000
    weight_decay = 0.005

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = BertSLU(15, 9)
print(f"Num of param:", sum(p.numel() for p in model.parameters()))
optimizer = torch.optim.AdamW(utils.weight_decay(model, config.weight_decay), lr=config.learning_rate, eps=config.adam_eps)
criterion = torch.nn.CrossEntropyLoss()

Num of param: 135016728


In [ ]:
trainer = Trainer(model, optimizer, criterion, amp=True, device=device)
# trainer.load_checkpoint("./checkpoint/checkpoint_bert.pt") Load checkpoint if wanna continue training
trainer.fit(train_loader, valid_loader, config.epochs, config.checkpoint_path)

In [ ]:
all_tokens, all_intents = trainer.test(test_loader)
len(all_tokens), len(all_intents)

 163 / 163


(1299, 1299)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
all_tokens[:5]

[[0, 5, 0, 6, 6, 6, 0, 5, 0, 8, 0, 0, 0, 0, 0, 0],
 [0, 5, 2, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 5, 5, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [0, 5, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0],
 [0, 5, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
all_seqs[:5]

['tăng ở cầu thang a mức tăng là 41',
 'tắt camera của huy đi cho tôi',
 'kiểm tra cho mình vào lúc 15 giờ 16 phút nhé',
 'đóng hộ anh cái cửa cuốn số 19',
 'bật cho mình cái laptop với mình cần làm việc']

In [ ]:
tokenizer.tokenize(all_seqs[0])

['tăng', 'ở', 'cầu', 'thang', 'a', 'mức', 'tăng', 'là', '41']

In [ ]:
all_intents[:5]

[7, 3, 9, 1, 6]

In [ ]:
INVERSE_MAP_TOKENS = {
    0: 'word',
    1: 'time at',
    2: 'device',
    3: 'changing value',
    4: 'scene',
    5: 'command',
    6: 'location',
    7: 'duration',
    8: 'target number'
 }

INVERSE_MAP_INTENTS = {
    0: 'Giảm độ sáng của thiết bị',
    1: 'Đóng thiết bị',
    2: 'Hủy hoạt cảnh',
    3: 'Tắt thiết bị',
    4: 'Tăng âm lượng của thiết bị',
    5: 'Giảm mức độ của thiết bị',
    6: 'Bật thiết bị',
    7: 'Tăng mức độ của thiết bị',
    8: 'Tăng nhiệt độ của thiết bị',
    9: 'Kiểm tra tình trạng thiết bị',
    10: 'Mở thiết bị',
    11: 'Giảm âm lượng của thiết bị',
    12: 'Kích hoạt cảnh',
    13: 'Giảm nhiệt độ của thiết bị',
    14: 'Tăng độ sáng của thiết bị'
}

In [ ]:
def collect_label(token):
    token = token[1:]
    for i in range(len(token) - 1, -1, -1):
        if token[i] != 0:
            token = token[:i+1]
            break
    token += [-1]
    map_labels = []
    cur = 0
    val = token[0]
    for idx, i in enumerate(token[1:], 1):
        if i == val:
            continue
        else:
            if val != 0:
                map_labels.append([cur, idx-1, val])
            val = i
            cur = idx
    return map_labels

def convert_into_output(all_tokens, all_intents, all_seqs, test_file_id, tokenizer):
    ans = []
    for idx in range(len(all_tokens)):
        token = all_tokens[idx]
        intent = all_intents[idx]
        seq = tokenizer.tokenize(all_seqs[idx])
        labels = collect_label(token)
        tmp_ans = {
            "intent": INVERSE_MAP_INTENTS[intent],
            "file": test_file_id[idx]
        }
        entities = []
        # print(labels)
        # print(seq)
        # return
        for label in labels:
            if label[-1] == 0:
                continue
            sub_text = seq[label[0]: label[1]+1]
            sub_text = tokenizer.decode(
                tokenizer.convert_tokens_to_ids(sub_text), skip_special_tokens=True
            )
            tmp_add = {"type": INVERSE_MAP_TOKENS[label[-1]], "filler": sub_text}
            check = list(filter(lambda x: tmp_add["type"] == x["type"] and tmp_add["filler"] == x["filler"], entities))
            if len(check):
                continue
            entities += [tmp_add]
        tmp_ans["entities"] = entities
        ans.append(tmp_ans)
        print("\r", end="")
        print(f"\r {idx+1} / {len(all_tokens)}", end="")
    return ans

In [ ]:
ans = convert_into_output(all_tokens, all_intents, all_seqs, test_file_id, tokenizer)

 1299 / 1299

In [ ]:
ans[:5]

[{'intent': 'Tăng mức độ của thiết bị',
  'file': 'TGZD2rv26WxpkV9LRAxwmVb.wav',
  'entities': [{'type': 'command', 'filler': 'tăng'},
   {'type': 'location', 'filler': 'cầu thang a'},
   {'type': 'target number', 'filler': '41'}]},
 {'intent': 'Tắt thiết bị',
  'file': 'QgEpeOnXqmu2gclgZxvGaVo.wav',
  'entities': [{'type': 'command', 'filler': 'tắt'},
   {'type': 'device', 'filler': 'camera của'},
   {'type': 'location', 'filler': 'huy'}]},
 {'intent': 'Kiểm tra tình trạng thiết bị',
  'file': 'OswACb2vqKb3OjMJkqveMn2.wav',
  'entities': [{'type': 'command', 'filler': 'kiểm tra'},
   {'type': 'time at', 'filler': '15 giờ 16 phút'}]},
 {'intent': 'Đóng thiết bị',
  'file': 'grFpJzV3KfLoCwTY8Wh2o9K.wav',
  'entities': [{'type': 'command', 'filler': 'đóng'},
   {'type': 'device', 'filler': 'cửa cuốn số 19'}]},
 {'intent': 'Bật thiết bị',
  'file': 'bwFuSjwUxFmpjSJTE7X9M2N.wav',
  'entities': [{'type': 'command', 'filler': 'bật'},
   {'type': 'device', 'filler': 'laptop'}]}]

In [ ]:
import json

with open("./submission.jsonl", "w", encoding="utf-8") as f:
    for line in ans:
        json.dump(line, f)
        f.write('\n')